In [2]:
import pandas as pd

In [ ]:
df_anac = pd.read_csv('anac.csv')
df_anac.info() 

In [5]:
def normalizacao(dataframe, colunas:[str], apelido: str):
    if len(colunas) == 1:
        normalizado = [[s, d] for s, d in enumerate(dataframe[colunas[0]].unique())]
        normalizado = pd.DataFrame(
            normalizado,
            columns = [
                apelido+'_sequence',
                'description'
            ]
        )
        dataframe = dataframe.merge(
            normalizado,
            left_on = colunas,
            right_on = 'description',
            how = 'left'
        )
        dataframe = dataframe.drop(
            columns = [*colunas, 'description']
        )
    else:
        normalizado = dataframe[colunas].drop_duplicates().reset_index()
        normalizado['index'] = normalizado.index
        normalizado = normalizado.rename(
            columns={'index': apelido+'_sequence'}
        )
        dataframe = dataframe.merge(
            normalizado, 
            right_on=colunas,
            left_on=colunas, 
            how='left'
        )
        for coluna in dataframe.columns:
            if coluna.endswith('_x') or coluna.endswith('_y') or coluna in colunas:
                dataframe = dataframe.drop(columns=coluna)        
    return dataframe, normalizado

In [11]:
anac, empresa = normalizacao(
    df_anac, 
    [
        'empresa_sigla',
        'empresa_nome', 
        'empresa_nacionalidade',
    ],
    'empresa'
    )

In [12]:
anac, aeroporto_origem = normalizacao(
    anac, 
    [
        'aeroporto_de_origem_sigla',
        'aeroporto_de_origem_nome',
        'aeroporto_de_origem_uf',
        'aeroporto_de_origem_regiao',
        'aeroporto_de_origem_pais',
        'aeroporto_de_origem_continente',
    ],
    'aeroporto_de_origem'
    )
anac, aeroporto_destino = normalizacao(
    anac, 
    [
        'aeroporto_de_destino_sigla',
        'aeroporto_de_destino_nome',
        'aeroporto_de_destino_uf',
        'aeroporto_de_destino_regiao',
        'aeroporto_de_destino_pais',
        'aeroporto_de_destino_continente',
    ],
    'aeroporto_de_destino'
    )
    

In [15]:
aeroporto_origem, endereco_origem = normalizacao(
    aeroporto_origem, 
    [
        'aeroporto_de_origem_uf',
        'aeroporto_de_origem_regiao',
        'aeroporto_de_origem_pais',
        'aeroporto_de_origem_continente',
    ],
    'endereco'
    )

In [20]:
endereco_origem = endereco_origem.rename(columns={
    'aeroporto_de_origem_uf':'uf',
    'aeroporto_de_origem_regiao':'regiao',
    'aeroporto_de_origem_pais':'pais',
    'aeroporto_de_origem_continente':'continente',
})

In [22]:
aeroporto_destino, endereco_destino = normalizacao(
    aeroporto_destino, 
    [
        'aeroporto_de_destino_uf',
        'aeroporto_de_destino_regiao',
        'aeroporto_de_destino_pais',
        'aeroporto_de_destino_continente',
    ],
    'endereco'
    )

endereco_destino = endereco_destino.rename(columns={
    'aeroporto_de_destino_uf':'uf',
    'aeroporto_de_destino_regiao':'regiao',
    'aeroporto_de_destino_pais':'pais',
    'aeroporto_de_destino_continente':'continente',
})

In [ ]:
colunas = ['uf', 'regiao', 'pais', 'continente']
endereco = endereco_origem.merge(
            endereco_destino, 
            right_on=colunas,
            left_on=colunas, 
            how='outer'
        )

In [30]:
# TA ERRADO ISSO AQUI
aeroporto_destino = aeroporto_destino.merge(
    endereco['endereco_sequence_y'],
    right_on=['endereco_sequence_y'],
    left_on=['endereco_sequence']
    )
aeroporto_destino = aeroporto_destino.drop(columns=['endereco_sequence'])
aeroporto_destino = aeroporto_destino.rename(columns={
    'endereco_sequence_y': 'endereco_sequence'
})

In [32]:
aeroporto_origem = aeroporto_origem.merge(
    endereco['endereco_sequence_x'],
    right_on=['endereco_sequence_x'],
    left_on=['endereco_sequence']
    )
aeroporto_origem = aeroporto_origem.drop(columns=['endereco_sequence'])
aeroporto_origem = aeroporto_origem.rename(columns={
    'endereco_sequence_x': 'endereco_sequence'
})

In [39]:
aeroporto_origem = aeroporto_origem.rename(columns={
    'aeroporto_de_origem_sequence': 'aeroporto_sequence',
    'aeroporto_de_origem_sigla': 'aeroporto_sigla',
    'aeroporto_de_origem_nome': 'aeroporto_nome',
})

In [41]:
aeroporto_destino = aeroporto_destino.rename(columns={
    'aeroporto_de_destino_sequence': 'aeroporto_sequence',
    'aeroporto_de_destino_sigla': 'aeroporto_sigla',
    'aeroporto_de_destino_nome': 'aeroporto_nome',
})

In [66]:
colunas = ['aeroporto_sigla', 'aeroporto_nome', 'endereco_sequence']
aeroporto = aeroporto_origem.merge(
            aeroporto_destino, 
            right_on=colunas,
            left_on=colunas, 
            how='outer',
            indicator=True
        )

In [71]:
aeroporto

,aeroporto_sequence,aeroporto_sigla,aeroporto_nome,endereco_sequence,aeroporto_de_destino_sequence,_merge
0,0.0,LFPO,ORLY (NEAR PARIS),0.0,NaN,left_only
1,19.0,LFPG,PARIS,0.0,NaN,left_only
2,77.0,LFBO,TOULOUSE,0.0,NaN,left_only
3,322.0,LFBT,TARBES,0.0,NaN,left_only
4,420.0,LFSB,"MULHOUSE, FRANCE/BASEL, SWITZERLAND",0.0,NaN,left_only
...,...,...,...,...,...,...
848,NaN,VMMC,TAIPA,124.0,480.0,right_only
849,NaN,GUCY,CONAKRY/GBESSIA,125.0,481.0,right_only
850,NaN,FYWH,WINDHOEK,126.0,482.0,right_only
851,NaN,LMML,LUQA,127.0,485.0,right_only
